# Basic visualisation of deformable with Paraview

Everything detailed in the *Basic visulation of rigids with Paraview* notebooks (located in *Tutorials/rigid/visu* directory) is still usefull. As such it will not be explained how to:

* load files,
* difference betwen _.pvd_ and stack of _.vtu_ files,
* the field choice and coloring,
* clipping, thresholding and the use of glyphs.

The type of files to load to looked into deformable mechanical bodies are **mecafe** ones.

Again, the first thing is to generate and simulate a simple case. Let us generate a simple case similar to *deformable/pre/2Dcontact* previously demonstrated:

### Generation

In [ ]:
# Initialization
from pathlib import Path
from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)


bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
sees   = pre.see_tables()
tacts  = pre.tact_behavs()

# 2D
dim = 2

# mod & mat
mo = pre.model(name='M2D_L',physics='MECAx',element='T3xxx',dimension=2, external_model='MatL_',
               kinematic='small',material='elas_',anisotropy='iso__',mass_storage='coher')
mods+=mo

ma = pre.material(name='steel',materialType='ELAS',elas='standard',
                  young=0.1e+15,nu=0.2,anisotropy='isotropic',
                  density=0.25e+4)

mats.addMaterial(ma)


#  meshing
m1 = pre.buildMesh2D('4T3', x0=0.1, y0=1.01, lx=0.8, ly=1.5, nb_elem_x=8, nb_elem_y=15)
m2 = pre.readMesh('../pre/gmsh_files/cheese.msh',2)

#avatar
b1 = pre.buildMeshedAvatar(m1,mo,ma)
b1.addContactors(group='down', shape='CLxxx', color='BLUEx')
bodies += b1

b2 = pre.buildMeshedAvatar(m2,mo,ma)
b2.imposeDrivenDof(group='down',component=[1,2],dofty='vlocy')
b2.addContactors(group='up', shape='ALpxx', color='BLUEx',reverse=True)
bodies += b2

# contact law
lclal = pre.tact_behav(name='gapc0', law='GAP_SGR_CLB', fric=0.3)
tacts+= lclal

# visibility table
vt = pre.see_table(CorpsCandidat   ='MAILx', candidat   ='CLxxx', colorCandidat   ='BLUEx',
                   CorpsAntagoniste='MAILx', antagoniste='ALpxx', colorAntagoniste='BLUEx',
                   behav=lclal,  alert=0.1, halo=0.2)
sees += vt


# post processing commands
post = pre.postpro_commands()
#
my_command = pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)

# Lets write
pre.writeDatbox(dim, mats, mods, bodies, tacts, sees, gravy=[0., -9.81, 0.], datbox_path='DATBOX')

### Computation

And now a simple computation. Again as with rigids, the only parameters of interset regarding the visualisation are the frequency at which the files must be written (usually called `freq_display`).

In [ ]:
# space dimension
dim = 2

# modeling hypothesis: 1 = plain strain
mhyp = 1

# time evolution parameters
dt = 1e-3
nb_steps = 500

# theta integrator parameter
theta = 0.5

# interaction parameters
Rloc_tol = 5.e-2

# nlgs parameters
tol = 1e-4
relax = 1.0
norm = 'Quad '
gs_it1 = 50
gs_it2 = 10
stype='Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

hfile = 'lmgc90.h5'

from pylmgc90 import chipy
from pylmgc90.chipy import computation

computation.initialize(dim, dt, theta, mhyp, h5_file=hfile, deformable=True)

for k in range(1, nb_steps+1):
    if k%50==0:
        print( f"computing step {k}")
    computation.one_step(stype, norm, tol, relax, gs_it1, gs_it2,
                         freq_write, freq_display)

computation.finalize()

### Files

There some files that can help to check the state of the contactors of the different meshes. These are:
* _ALpxx.vtu_
* _CLxxx.vtu_
* _ASpxx.vtu_
* _CSpxx.vtu_

Of course these files are present only if contactors of the respecive type has been found.
As previously pointed out, the interesting files are the **mecafe** ones.

For a deformable object the default node fields values are:
* _Disp_ displacement vector
* _E_ the deformation tensor (stored as a vector of 9 components)
* _E vol_ the deformation energy scalar value
* _Fdyn_ the inertia part of the internal forces vector (dynamic)
* _Fext_ the external volumic forces vector (usually gravity)
* _Fint_ the internal forces vector
* _Ids_ the integer id of the body
* _Node Id_ the integer id of each node
* _Reac_ the reaction forces vector
* _Res_ the residu, used in non linear loop computation
* _S_ the stress tensore (stored as a vector of 9 components)
* _Svm_ the Von Mises stress value
* _Velocy_ the velocity vector

There is only one value by default attached to the element instead of the node is the _element id_ field.
![paraview_default.png](./images/paraview_default.png)
<br>


### Warp by vector

The meshes are always displayed in their initial configuration. So up until now, if nothing has been done, the user can see the changes inside the lower mesh but not see any movement.

This where the *Warp by vector* button comes into play. It allows to deforme the mesh according to a field. In our case the *Disp* one. It can be noted that, in case of very small deformation, a scale factor can be provided to amplify the movement; usually 1. is the most suited value :

![paraview_warp.png](./images/paraview_warp.png)
<br>
